In [8]:
import re
# import emoji
import string
import pymorphy3

import pandas as pd

______________________________________________________________________________________________________________________________________________________________________________________

Подгатовка данных

In [9]:
data = pd.read_csv('date/annotated_data.csv')

data.drop(columns=['categories'], inplace=True)

morph = pymorphy3.MorphAnalyzer()
stop_words = set(['и', 'в', 'на', 'с', 'чем', 'как', 'что', 'г'])                                                            # Список стоп слов

def clean_text(text):

    punctuation = string.punctuation

    cleaned_text = text.translate(str.maketrans('', '', punctuation))
    cleaned_text = re.sub(r"[A-Za-z0-9]", '', cleaned_text)

    # Удаляем лишние пробелы
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()

    cleaned_text = cleaned_text.lower()                                 # Приводит текст к нижнему регистру
    cleaned_text= cleaned_text.strip()                                  # Удаляет начальные и конечные пробелы

    tokens = cleaned_text.split()                                                                                               # Токинезация

    Lemmatization = [morph.parse(token)[0].normal_form for token in tokens]                                             # Лемматизация

    filtered_words = [Lemmatization for Lemmatization in Lemmatization if Lemmatization.lower() not in stop_words]      # Удаление стоп-слов

    result = ' '.join(filtered_words)                                                                                   # Удаление лишних пробелов

    return result

data['comment'] = data['comment'].apply(clean_text)

data = data[~data.duplicated('comment')]

data = data.dropna(subset=['comment']).loc[data['comment'] != '']

data.to_csv('date/processed_data.csv', index=False)